In [1]:
from keras.models import Sequential
from keras.models import model_from_json
from keras import backend as K
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
import os

# Load existing model.
with open("ResNet50-UdacityRealandSimMix-Best-val_acc-1.0.json",'r') as f:
    modelJSON = f.read()

model = model_from_json(modelJSON)
model.load_weights("ResNet50-UdacityRealandSimMix-Best-val_acc-1.0.h5")
print(model.summary())

print([node.op.name for node in model.outputs])

C:\Users\tknight\AppData\Roaming\Python\Python36\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
predictions (Dense)          (None, 4)                 4100      
Total para

In [2]:
# All new operations will be in test mode from now on.
K.set_learning_phase(0)

# Serialize the model and get its weights, for quick re-building.
config = model.get_config()
weights = model.get_weights()

# Re-build a model where the learning phase is now hard-coded to 0.
# new_model = Sequential.from_config(config)
# new_model.set_weights(weights)

temp_dir = "graph"
checkpoint_prefix = os.path.join(temp_dir, "saved_checkpoint")
checkpoint_state_name = "checkpoint_state"
input_graph_name = "input_graph.pb"
output_graph_name = "output_graph.pb"

# Temporary save graph to disk without weights included.
saver = tf.train.Saver()
checkpoint_path = saver.save(K.get_session(), checkpoint_prefix, global_step=0, latest_filename=checkpoint_state_name)
tf.train.write_graph(K.get_session().graph, temp_dir, input_graph_name)

input_graph_path = os.path.join(temp_dir, input_graph_name)
input_saver_def_path = ""
input_binary = False
output_node_names = "predictions/Softmax" # model dependent
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_graph_path = os.path.join(temp_dir, output_graph_name)
clear_devices = False

# Embed weights inside the graph and save to disk.
freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path,
                          output_node_names, restore_op_name,
                          filename_tensor_name, output_graph_path,
                          clear_devices, "")

INFO:tensorflow:Restoring parameters from graph\saved_checkpoint-0
INFO:tensorflow:Froze 322 variables.
Converted 322 variables to const ops.


In [3]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [4]:
    graph = load_graph(output_graph_path)

    # We can verify that we can access the list of operations in the graph
    for op in graph.get_operations():
        print(op.name)
        # prefix/image_input
        # ...
        # prefix/predictions/Softmax

prefix/image_input
prefix/conv1/kernel
prefix/conv1/kernel/read
prefix/conv1/bias
prefix/conv1/bias/read
prefix/bn_conv1/gamma
prefix/bn_conv1/gamma/read
prefix/bn_conv1/beta
prefix/bn_conv1/beta/read
prefix/bn_conv1/moving_mean
prefix/bn_conv1/moving_mean/read
prefix/bn_conv1/moving_variance
prefix/bn_conv1/moving_variance/read
prefix/bn_conv1/keras_learning_phase/input
prefix/bn_conv1/keras_learning_phase
prefix/res2a_branch2a/kernel
prefix/res2a_branch2a/kernel/read
prefix/res2a_branch2a/bias
prefix/res2a_branch2a/bias/read
prefix/bn2a_branch2a/gamma
prefix/bn2a_branch2a/gamma/read
prefix/bn2a_branch2a/beta
prefix/bn2a_branch2a/beta/read
prefix/bn2a_branch2a/moving_mean
prefix/bn2a_branch2a/moving_mean/read
prefix/bn2a_branch2a/moving_variance
prefix/bn2a_branch2a/moving_variance/read
prefix/res2a_branch2b/kernel
prefix/res2a_branch2b/kernel/read
prefix/res2a_branch2b/bias
prefix/res2a_branch2b/bias/read
prefix/bn2a_branch2b/gamma
prefix/bn2a_branch2b/gamma/read
prefix/bn2a_branch2b

prefix/dropout_1/cond/dropout/random_uniform
prefix/dropout_1/cond/dropout/add
prefix/dropout_1/cond/dropout/Floor
prefix/dropout_1/cond/dropout/div
prefix/dropout_1/cond/dropout/mul
prefix/dropout_1/cond/Switch_1
prefix/dropout_1/cond/Merge
prefix/fc1/kernel
prefix/fc1/kernel/read
prefix/fc1/bias
prefix/fc1/bias/read
prefix/fc1/MatMul
prefix/fc1/BiasAdd
prefix/fc1/Relu
prefix/dropout_2/cond/Switch
prefix/dropout_2/cond/switch_t
prefix/dropout_2/cond/pred_id
prefix/dropout_2/cond/mul/y
prefix/dropout_2/cond/mul
prefix/dropout_2/cond/mul/Switch
prefix/dropout_2/cond/dropout/keep_prob
prefix/dropout_2/cond/dropout/Shape
prefix/dropout_2/cond/dropout/random_uniform/min
prefix/dropout_2/cond/dropout/random_uniform/max
prefix/dropout_2/cond/dropout/random_uniform/RandomUniform
prefix/dropout_2/cond/dropout/random_uniform/sub
prefix/dropout_2/cond/dropout/random_uniform/mul
prefix/dropout_2/cond/dropout/random_uniform
prefix/dropout_2/cond/dropout/add
prefix/dropout_2/cond/dropout/Floor
pref

In [7]:
    import cv2
    import numpy as np
    from keras.applications import resnet50
    
    labels = np.array(['green', 'none', 'red', 'yellow'])
    
    # We access the input and output nodes 
    x = graph.get_tensor_by_name('prefix/image_input:0')
    y = graph.get_tensor_by_name('prefix/predictions/Softmax:0')
    
    image = cv2.imread('0_resized.png')
    image = cv2.resize(image, (224, 224))
    cv2.imshow('image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    np_image_data = np.asarray(image)
    np_final = np.expand_dims(np_image_data, axis=0)
    np_final = resnet50.preprocess_input(np_final.astype('float64'))
    
    # We launch a Session
    with tf.Session(graph=graph) as sess:
        yhat = sess.run(y, feed_dict={x: np_final})
    
    yhat = yhat[0]
    y_class = yhat.argmax(axis=-1)
    
    print("Detected, {:d} = {} with probability = {:4.2f} %.".format(y_class, labels[y_class], yhat[y_class]*100))

Detected, 0 = green with probability = 98.01 %.
